In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.model_selection import train_test_split
import sklearn.neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics
from sklearn.preprocessing import scale
from collections import Counter
# Import dataset
df=pd.read_csv('C:/Users/Blank Panther/Downloads/iris (1).csv')
df.head()

,Sepal length,Sepal width,Petal length,Petal width,Species
0,4.5,2.3,1.3,0.3,setosa
1,5.7,4.4,1.5,0.4,setosa
2,7.9,3.8,6.4,2.0,virginica
3,7.7,3.8,6.7,2.2,virginica
4,7.7,2.6,6.9,2.3,virginica


In [2]:
df.shape

(150, 5)

In [3]:
# Splitting data into 70:30 train:test ratio
df_X=df.iloc[:,:4]
df_Y=df.iloc[:,4]
print('X', df_X)
print('Y',df_Y)
X_train,X_test,Y_train,Y_test=train_test_split(df_X,df_Y,test_size=0.3,random_state=33)

X      Sepal length  Sepal width  Petal length  Petal width
0             4.5          2.3           1.3          0.3
1             5.7          4.4           1.5          0.4
2             7.9          3.8           6.4          2.0
3             7.7          3.8           6.7          2.2
4             7.7          2.6           6.9          2.3
5             5.5          4.2           1.4          0.2
6             5.0          2.0           3.5          1.0
7             7.2          3.6           6.1          2.5
8             5.8          4.0           1.2          0.2
9             7.7          2.8           6.7          2.0
10            5.2          4.1           1.5          0.1
11            4.4          2.9           1.4          0.2
12            5.8          2.8           5.1          2.4
13            4.3          3.0           1.1          0.1
14            7.7          3.0           6.1          2.3
15            6.0          3.4           4.5          1.6
16          

In [4]:
# Changing the index of the records to sequential

In [5]:
X_train.index=range(len(X_train))
Y_train.index=range(len(X_train))
X_test.index=range(len(X_test))
Y_test.index=range(len(Y_test))
print('X_train.index',X_train.index)
print('Y_train.index',Y_train.index)
print('X_test.index',X_test.index)
print('Y_test.index',Y_test.index)

X_train.index RangeIndex(start=0, stop=105, step=1)
Y_train.index RangeIndex(start=0, stop=105, step=1)
X_test.index RangeIndex(start=0, stop=45, step=1)
Y_test.index RangeIndex(start=0, stop=45, step=1)


In [6]:
# Function to return the list of distances of the test records from train records
def distNeighbours(X_train,Y_train,X_test,K):
    distance=[]
    for i in range(len(X_train)):
        eDistance=0
        for j in range(len(X_train.columns)):   
                eDistance+=round(np.sqrt(pow((X_train.iloc[i,j]-X_test[j]),2)),2)
        distance.append((eDistance,i,Y_train.iloc[i]))
        distance=sorted(distance, key=lambda x: x[0])[0:K]
    return distance

In [7]:
# Predict the output of the categorical variables based on K nearest neighbours 
# Output is the most frequent class among K nearest neighbours
def predictOutputCategorical(X_train,Y_train,X_test,K):
    neighbours=[]
    responses=[]
    for i in range(len(X_test)):
        neighbours.append(distNeighbours(X_train,Y_train,X_test.iloc[i,:],K))
    for i in neighbours:
        votes={}
        for j in i:
            if j[-1] in votes.keys():
                votes[j[-1]]=votes[j[-1]]+1
            else:
                votes[j[-1]]=1
        responses.append(sorted(votes,key=votes.get,reverse=True)[0])
    return responses
# Predict the output of the numeric variables based on K nearest neighbours
# Output is the mean of the K nearest neighbours
def predictOutputNumeric(X_train,Y_train,X_test,K):
    neighbours=[]
    responses=[]
    for i in range(len(X_test)):
        neighbours.append(distNeighbours(X_train,Y_train,X_test.iloc[i,:],K))
    for i in neighbours:
        mean=0
        for j in i:
            mean+=j[-1]
        mean=mean/K
        responses.append(mean)
    return responses
# Accuarcy of the categorical predictions 
def getAccuracyCategorical(actual,predicted):
    correct=0
    for i in range(len(predicted)):
        if predicted[i]==actual[i]:
            correct+=1
    return round((correct/len(actual))*100,2)
# Accuarcy of the numerical predictions
def getAccuracyNumeric(actual,predicted):
    error=0
    for i in range(len(predicted)):
        error+=pow((actual[i]-predicted[i]),2)
    error=error/len(predicted)-1
    return 100-error


Accuracy

In [8]:
#Predict species
output=predictOutputCategorical(X_train,Y_train,X_test,3)
getAccuracyCategorical(Y_test,output)

95.56

In [9]:
# Fit model using in built sklearn function
model=KNeighborsClassifier(n_neighbors=3,p=2,metric='minkowski')
model.fit(X_train,Y_train)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [10]:
#Accuracy of the model
print('Accuracy: {:^0.2f}'.format(metrics.accuracy_score(Y_test,model.predict(X_test))*100))

Accuracy: 95.56


In [11]:
# Check whether the both outputs are same or not
# They are same as displayed below
output==model.predict(X_test)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])